In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

# Load sample data
data = pd.read_csv("final pro.csv")
df = pd.read_csv("final pro.csv")
df_table =pd.read_csv("tabledashdata.csv")

# Initialize Dash app
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.SUPERHERO])
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 48,
    "left": 0,
    "bottom": 0,
    "width": "32rem",
    "padding": "2rem 1rem",
    "background-color": "#0f2537",
}

# padding for the page content
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H4("Inflation"),
        html.P("Inflation is the rate of increase in prices over a given period of time. Inflation is typically a broad measure, such as the overall increase in prices or the increase in the cost of living in a country."),
        html.H4("CPI and WPI"),
        html.P('CPI stands for Consumer Price Index, which is a measure of the average change in prices of goods and services consumed by households. On the other hand, WPI stands for Wholesale Price Index, which measures the average change in prices of goods sold in bulk quantity by wholesalers. Both indices are used to measure inflation rates in an economy, but they differ in terms of the types of goods and services they measure and the level of the market they represent.'),
        html.H4('Unemployment'),
        html.P('Unemployment is when an individual who is not employed and is seeking employment, cannot find work. Unemployment is a key indicator of the health of an economy. A low unemployment rate represents a strong economy while a high unemployment rate represents a weak economy.'),
    ],
    style=SIDEBAR_STYLE,
)

# Define layout
app.layout = html.Div([sidebar,
    # Title
    html.H1('Inflation Dashboard', style={'color': 'black','background-color':'#88d8b0'}),
    
     html.Div([
         html.H2('Year Wise Inflation',style={'position':'relative','left':'550px'}),
    # Add a dropdown to choose the chart type
    html.Div([
    dcc.Dropdown(
        id='chart-type',
        options=[
            {'label': 'Bar Chart', 'value': 'bar'},
            {'label': 'Line Chart', 'value': 'line'}
        ],
        value='bar',
        style={'width': '300px','color': 'black','background-color':'#88d8b0','float':'right'}
    ),
    html.Div([
        html.Button('5 Years', id='button-5-years', style={'width': '120px', 'height': '35px','font-size':'20px'}),
        html.Button('3 Years', id='button-3-years', style={'width': '120px', 'height': '35px','font-size':'20px'}),
        html.Button('Entire Years', id='button-all-years', style={'width': '170px', 'height': '35px','font-size':'20px'})
    ], style={'display': 'inline','position': 'relative', 'left': '750px'})
]),
   
    # Add the chart
    dcc.Graph(id='my-chart',style={'position':'relative','left':'550px','top':'30px'}),
   
    # Add a tag that updates when a year range button is clicked
    html.Div(id='button-output')
]),
   #stacked
    html.Div([
   
    # Title
    html.Br(),
    html.Br(),
    html.Br(),
    html.H2('Year Wise Inflation And Unemployment',style={'position':'relative','left':'550px'}),
    html.Br(),
   
    # Dropdown to select chart type
    dcc.Dropdown(
        id='chart-type2',
        options=[
            {'label': 'Stacked Bar Chart', 'value': 'stacked'},
            {'label': 'Dual Axis Chart', 'value': 'dual'}
        ],
        value='stacked',
        style={'color': 'black','background-color':'#88d8b0','width':'400px','float':'right'}
    ),
        html.Br(),
   
    # Range slider to select date range
    html.Div([
    dcc.RangeSlider(
    id='date-range',
    min=df['Year'].min(),
    max=df['Year'].max(),
    value=[df['Year'].min(), df['Year'].max()],
    marks={str(date): {'label' : str(date), 'style':{'color':'white','width':'20%'}} for date in range(1991,2020,3)},
    )],
        style={'position':'absolute','left':'550px','width':'550px','top':'830px'}
        ),
        html.Br(),
        html.Br(),
    # Graph
    dcc.Graph(id='chart',style={'position':'absolute','left':'550px','width':'950px','height':'500px'})
]),
    #table
   dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in df_table.columns],
    data=df_table.to_dict('records'),
    style_header={
        'backgroundColor': 'rgb(210, 210, 210)',
        'color': 'black',
        'fontWeight': 'bold'
    },
    style_data={
        'color': 'black',
        'backgroundColor': 'white'
    },
    style_table={
        'maxHeight': '400px',
        'maxWidth':'500px',
        'overflowY': 'scroll',
        'width': '100%',
        'float':'right',
        'top':'580px'
    },
    style_cell={
        'minWidth': '50px', 'maxWidth': '100px',
        'whiteSpace': 'normal',
        'textOverflow': 'ellipsis',
    },
)
   
])

# Define the callbacks for the app
@app.callback(
    [Output('my-chart', 'figure'), Output('button-output', 'children')],
    [
        Input('chart-type', 'value'),
        Input('button-5-years', 'n_clicks'),
        Input('button-3-years', 'n_clicks'),
        Input('button-all-years', 'n_clicks')
    ],
    [State('button-output', 'children')]
)
def update_chart(chart_type, clicks_5_years, clicks_3_years, clicks_all_years, button_output):
    # Determine which button was clicked
    ctx = dash.callback_context
    if not ctx.triggered:
        button_id = 'button-all-years'
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    if button_id == 'button-5-years':
        start_year = data['Year'].max() - 4
    elif button_id == 'button-3-years':
        start_year = data['Year'].max() - 2
    else:
        start_year = data['Year'].min()
   
    # Filter the data to include only the selected years
    filtered_data = data[data['Year'] >= start_year]
   
    # Create the chart based on the selected chart type
    if chart_type == 'bar':
        fig = px.bar(filtered_data, x='Year', y='Inflation',title='Year Wise Inflation',color_discrete_sequence=['#edac3b'])
    else:
        fig = px.line(filtered_data, x='Year', y='Inflation',title='Year Wise Inflation',markers=True,color_discrete_sequence=['red'])
       
        # Modify the size of the chart
    fig.update_layout(height=550, width=950,plot_bgcolor='#0f2537',
                      paper_bgcolor='#0f2537',titlefont=dict(color='white'),
                      title= {'text': 'This Chart Shows Year Wise Inflation', 'x': 0.55})
    fig.update_xaxes(title_font=dict(color='white'),tickfont=dict(color='white'))
    fig.update_yaxes(title_font=dict(color='white'),tickfont=dict(color='white'))

   
    return fig, button_output
# Define Callbacks
@app.callback(
    dash.dependencies.Output('chart', 'figure'),
    [dash.dependencies.Input('chart-type2', 'value'),
     dash.dependencies.Input('date-range', 'value')])
def update_chart(chart_type, date_range):
    filtered_df = df[(df['Year'] >= date_range[0]) & (df['Year'] <= date_range[1])]
   
    if chart_type == 'stacked':
        data = data = [
            go.Bar(x=filtered_df['Year'], y=filtered_df['unemployment'], name='Unemployement', marker=dict(color='#8a1e1e')),
            go.Bar(x=filtered_df['Year'], y=filtered_df['Inflation'], name='Inflation', marker=dict(color='#1a7333'))]
        layout = go.Layout(barmode='stack',
                           title="Year Wise Inflation and Unemployment",
                           plot_bgcolor='#0f2537',
                          paper_bgcolor='#0f2537',
                           titlefont=dict(color='white'),
                         xaxis=dict(title='Years', titlefont=dict(color='white'), tickfont=dict(color='white')),
                           yaxis=dict(title='Unemployment & Inflation', titlefont=dict(color='white'), tickfont=dict(color='white')))
       
    elif chart_type == 'dual':
        data = [
            go.Scatter(x=filtered_df['Year'], y=filtered_df['unemployment'], name='Unemployment',mode='lines+markers', line=dict(color='#8a1e1e'), marker=dict(color='red')),
            go.Scatter(x=filtered_df['Year'], y=filtered_df['Inflation'], name='Inflation', yaxis='y2',mode='lines+markers', line=dict(color='#1a7333'), marker=dict(color='green'))
        ]
        layout = go.Layout(
            xaxis=dict(title='Years',titlefont=dict(color='white'),tickfont=dict(color='white')),
            yaxis=dict(title='Unemployment & Inflation',titlefont=dict(color='white'),tickfont=dict(color='white')),
            yaxis2=dict(overlaying='y', side='right',titlefont=dict(color='white'),tickfont=dict(color='white')),
            title='Year Wise Inflation and Unemployment',
            plot_bgcolor='#0f2537',
            paper_bgcolor='#0f2537',
            titlefont=dict(color='white')
        )
   
    return {'data': data, 'layout': layout}

if __name__ == '__main__':
    app.run_server(debug=False)

C:\Users\kavin\AppData\Local\Temp\ipykernel_10644\3812325906.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\kavin\AppData\Local\Temp\ipykernel_10644\3812325906.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Dash is running on http://127.0.0.1:8050/



C:\Users\kavin\AppData\Local\Temp\ipykernel_10644\3812325906.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Apr/2023 19:00:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 19:00:48] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 19:00:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 19:00:48] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 19:00:48] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Apr/2023 19:00:48] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Apr/2023 19:00:48] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 19:00:48] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Apr/2023 19:00:48] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 19:00:49] "POST /_dash-update-component HT